In [3]:
import streamlit as st
import joblib
import pandas as pd
import yfinance as yf
import numpy as np

# --- Helper Functions (Must replicate feature engineering) ---
@st.cache_data
def get_latest_data(ticker):
    # Fetch the last 60 days of data to ensure enough history for 30-day MA
    data = yf.download(ticker, period="60d", interval="1d")
    return data

def prepare_features_for_prediction(data):
    """
    Calculates the exact features the model was trained on,
    using the last row of the historical data.
    """
    df_copy = data[['Close']].copy()

    # Replicate Lag and SMA features exactly as in training
    lag_days = [1, 7, 30]
    for day in lag_days:
        df_copy[f'Lag_{day}d'] = df_copy['Close'].shift(day)

    df_copy['SMA_7'] = df_copy['Close'].rolling(window=7).mean()
    df_copy['SMA_30'] = df_copy['Close'].rolling(window=30).mean()

    # Get the features from the very last (current) day
    # This row has all the features, but a NaN for 'Target' (which is tomorrow's price)
    latest_features = df_copy.iloc[-1].drop('Close') # Drop 'Close' from features

    # Convert to a DataFrame row format that the model expects
    features_df = latest_features.drop('Target').to_frame().T
    return features_df

# --- Streamlit Application ---
st.title("₿ Crypto Price Predictor (Random Forest)")
st.markdown("Predicting tomorrow's closing price for BTC and ETH.")
st.markdown("---")

# Load Models
try:
    rf_btc = joblib.load("rf_btc_predictor.pkl")
    rf_eth = joblib.load("rf_eth_predictor.pkl")
except FileNotFoundError:
    st.error("Model files not found. Please run Step 3 (Training) first.")
    st.stop()

# --- Prediction Logic for BTC ---
st.header("Bitcoin (BTC) Prediction")
btc_data_latest = get_latest_data("BTC-USD")
btc_input_features = prepare_features_for_prediction(btc_data_latest)

btc_prediction = rf_btc.predict(btc_input_features)[0]
current_btc_price = btc_data_latest['Close'].iloc[-1]
delta_btc = btc_prediction - current_btc_price

st.metric(
    label=f"Predicted Close Price for Tomorrow (BTC)",
    value=f"${btc_prediction:,.2f}",
    delta=f"{delta_btc:,.2f} ({delta_btc/current_btc_price*100:.2f}%)"
)

st.caption(f"Current BTC Price: ${current_btc_price:,.2f}")

st.markdown("---")

# --- Prediction Logic for ETH ---
st.header("Ethereum (ETH) Prediction")
eth_data_latest = get_latest_data("ETH-USD")
eth_input_features = prepare_features_for_prediction(eth_data_latest)

eth_prediction = rf_eth.predict(eth_input_features)[0]
current_eth_price = eth_data_latest['Close'].iloc[-1]
delta_eth = eth_prediction - current_eth_price

st.metric(
    label=f"Predicted Close Price for Tomorrow (ETH)",
    value=f"${eth_prediction:,.2f}",
    delta=f"{delta_eth:,.2f} ({delta_eth/current_eth_price*100:.2f}%)"
)

st.caption(f"Current ETH Price: ${current_eth_price:,.2f}")

2025-11-10 21:17:48.593 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-11-10 21:17:48.596 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 21:17:48.746 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-11-10 21:17:48.747 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 21:17:48.750 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 21:17:48.751 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 21:17:48.752 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 21:17:48.754 Thread 'MainThread': miss

KeyError: 'Target'

In [2]:
%pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 58.0 MB/s eta 0:00:00
